In [1]:
import codecs
import nltk.data
# nltk.download('punkt')
import fileinput
import re
import random
import time
import pickle
import cPickle
import itertools
from textblob.classifiers import DecisionTreeClassifier

In [2]:
# split sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
with codecs.open("book_1.txt",'r',encoding='utf8') as f:
    data = f.read()
    script = '\n'.join(tokenizer.tokenize(data))
    script = script.encode("utf8")
with open("book_1_1.txt","w") as f:
    f.write(script)

In [6]:
# generate name pairs and sentences including pairs
# remove name pairs and pucntuations from the sentence
# input
# txt: each line contains a sentence
# namelist: names for generating pairs
# namedict: containing all nick names
# familylist: for labeling namepairs
# output
# namepairs
# sentences comtaining namepairs
# pairlabel for each pair
def senAndname(txtpath, namelist, namedict, familylist):
    nicklist = list(itertools.chain(*[namedict[name] for name in namelist]))
    namepairs = []
    sentences = []
    pairlabel = []
    for line in fileinput.input(txtpath):
        # lower case for comparision
        line1 = re.findall(r"[\w']+", line.lower())
        # original for feature
        line2 = re.findall(r"[\w']+", line)
        for x in namelist:
            for y in namelist[(namelist.index(x)+1):]:
                occur = 0
                for name1, name2 in itertools.product(namedict[x], namedict[y]):
                    if " " in name1 or " " in name2:
                        if set(name1.lower().split()).intersection(line1) ==  set(name1.lower().split()) and set(name2.lower().split()).intersection(line1) ==  set(name2.lower().split()):   
                            occur += 1
                            line3 = [w for w in line2 if not w in nicklist + name1.split() + name2.split()]
                    else:
                        if name1.lower() in line1 and name2.lower() in line1:
                            occur += 1
                            line3 = [w for w in line2 if not w in nicklist]
                if occur:
                    line3 = " ".join(line3)
                    namepairs.append((x,y))
                    sentences.append(str(line3))
                    if x in familylist and y in familylist:
                        pairlabel.append("pos")
                    else:
                        pairlabel.append("neg")
    fileinput.close()
    return([namepairs, sentences, pairlabel])

In [9]:
# read name dict
import pickle
with open("nick_ep1", "rb") as f:
    ep1 = pickle.load(f)

In [10]:
# data sets   
Infamily_train = ["Molly Weasley", "Arthur Weasley", "Ronald Weasley"]
Outfamily_train = ["Harry Potter", "Albus Dumbledore", "Severus Snape", "Hermione Granger"]
total_train = Infamily_train + Outfamily_train
Infamily = ["Molly Weasley", "Arthur Weasley", "Ronald Weasley", "Charlie Weasley", 
            "George Weasley", "Ginevra Weasley", "Percy Weasley", "William Weasley", "Fred Weasley"]
total = ep1.keys()

In [6]:
train = senAndname("book_1_1.txt", total_train, ep1, Infamily_train)

In [11]:
test = senAndname("book_1_1.txt", total, ep1, Infamily)

In [10]:
with open("train", "wb") as f:
    pickle.dump(train, f)
with open("test", "wb") as f:
    pickle.dump(test, f)

In [30]:
# # sentense, labels and name pair
# sample3 = list(zip(script4, label, name_pair))
# random.seed(1)
# sample = random.sample(sample3, 100)
# # sentense and labels
# sample2 = []
# for x in sample3:
#     y = (x[0], x[1])
#     sample2.append(y)
# # train: sentense and labels| train_name_pair
# train = []
# train_name_pair = []
# for x in sample:
#     y = (x[0], x[1])
#     train.append(y)
#     train_name_pair.append(x[2])

# # test3: sentense, lables and test_name_ pair
# test3 = [x for x in sample3 if x not in sample]
# # test2: sentense and labels for test
# # isolated test name pair
# test2 = []
# test_name_pair = []
# for x in test3:
#     test_name_pair.append(x[2])
#     y = (x[0], x[1])
#     test2.append(y)
# # isolated test label
# test_label = []
# for x in test3:
#     test_label.append(x[1])
# test_label1 = "\n".join(test_label)
# with open("test_label.txt", "w") as f:
#     f.write(test_label1)

In [28]:
# train classifier
traindata = zip(train[1],train[2])
start_time = time.time()
cl_tree = DecisionTreeClassifier(traindata)
print("--- %s seconds ---" % (time.time() - start_time))

--- 180.143886089 seconds ---


In [29]:
# plot decision tree
print(cl_tree.pretty_format())

contains(twins)=False? ................................ neg
  contains(Fred)=False? ............................... neg
    contains(Charlie)=False? .......................... neg
      contains(sweater)=False? ........................ neg
      contains(sweater)=True? ......................... pos
    contains(Charlie)=True? ........................... pos
      contains(about)=False? .......................... pos
      contains(about)=True? ........................... neg
  contains(Fred)=True? ................................ pos
    contains(points)=False? ........................... pos
    contains(points)=True? ............................ neg
contains(twins)=True? ................................. pos



In [32]:
# save classifier
with open("cl_tree", "wb") as f:
    cPickle.dump(cl_tree, f)

In [39]:
# load classifier
with open("cl_tree", "rb") as f:
    cl_tree = cPickle.load(f)

In [42]:
# classification
start_time = time.time()
label_tree = []
feature_tree = []
for x in test[1]:
    l = cl_tree.classify(x)
    f = cl_tree.extract_features(x)
    label_tree.append(l)
    feature_tree.append(f)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2439.38496995 seconds ---


In [49]:
with open("label_tree.pkl","wb") as f:
    pickle.dump(label_tree, f)
with open("feature_tree.pkl","wb") as f:
    pickle.dump(feature_tree, f)

In [2]:
with open("label_tree.pkl","rb") as f:
    label_tree = pickle.load(f)

In [51]:
label_tree.count("neg")

2672

In [52]:
pred = zip(test[0], label_tree, test[2])
# in pred_label value, first element is accumulated prediction, the second element is total count of sentences for the 
# pairs, the third one is real relation
tree_pred = {}
for x in pred:
    if x[0] in tree_pred.keys():
        tree_pred[x[0]][1] += 1
    else:
        tree_pred[x[0]] = [0,1,0]
    if x[1] == "pos":
        tree_pred[x[0]][0] += 1
    else:
        tree_pred[x[0]][0] -= 1
    if x[2] == "pos":
        tree_pred[x[0]][2] = 1
    else:
        tree_pred[x[0]][2] = 0

In [53]:
tree_pred

{('Albus Dumbledore', 'Dudley Dursley'): [-1, 1, 0],
 ('Albus Dumbledore', 'Godric Gryffindor'): [-1, 1, 0],
 ('Albus Dumbledore', 'Marjorie Dursley'): [-1, 1, 0],
 ('Albus Dumbledore', 'Minerva McGonagall'): [-9, 9, 0],
 ('Albus Dumbledore', 'Molly Weasley'): [1, 1, 0],
 ('Albus Dumbledore', 'Ronald Weasley'): [-4, 6, 0],
 ('Albus Dumbledore', 'Rubeus Hagrid'): [-8, 10, 0],
 ('Alicia Spinnet', 'Molly Weasley'): [-1, 1, 0],
 ('Alicia Spinnet', 'Ronald Weasley'): [-1, 1, 0],
 ('Angelina Johnson', 'Alicia Spinnet'): [-1, 1, 0],
 ('Angelina Johnson', 'Charlie Weasley'): [-1, 1, 0],
 ('Angelina Johnson', 'George Weasley'): [-1, 1, 0],
 ('Angelina Johnson', 'Godric Gryffindor'): [-2, 2, 0],
 ('Angelina Johnson', 'Katie Bell'): [-1, 1, 0],
 ('Angelina Johnson', 'Marcus Flint'): [-1, 1, 0],
 ('Angelina Johnson', 'Molly Weasley'): [-1, 1, 0],
 ('Angelina Johnson', 'Oliver Wood'): [-1, 1, 0],
 ('Angelina Johnson', 'Percy Weasley'): [-1, 1, 0],
 ('Angelina Johnson', 'Ronald Weasley'): [-1, 1, 0]

In [12]:
# accuracy
j = 0
for i in range(len(test[2])):
    if test[2][i] != label_tree[i]:
        j += 1
print(j/float(i))
print(j)

0.264705882353
1107


In [14]:
len(test[2])

4183